In [1]:
stock = 'ORCL'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    48.474823
2018-07-24    48.474823
2018-07-25    48.753704
2018-07-26    48.674023
2018-07-27    48.434986
2018-07-30    47.538597
2018-07-31    47.488796
2018-08-01    47.160118
2018-08-02    47.707916
2018-08-03    48.275631
2018-08-06    48.474823
2018-08-07    48.285587
2018-08-08    48.195950
2018-08-09    48.345348
2018-08-10    48.126228
2018-08-13    47.817471
2018-08-14    48.036591
2018-08-15    47.648155
2018-08-16    47.907112
2018-08-17    48.166069
2018-08-20    48.225826
2018-08-21    48.215870
2018-08-22    48.604305
2018-08-23    48.763664
2018-08-24    49.062458
2018-08-27    49.132183
2018-08-28    48.444946
2018-08-29    48.693943
2018-08-30    48.185989
2018-08-31    48.385189
                ...    
2018-09-18    48.833382
2018-09-19    49.231777
2018-09-20    50.227768
2018-09-21    50.895081
2018-09-24    51.034523
2018-09-25    51.512596
2018-09-26    51.422958
2018-09-27    51.482716
2018-09-28    51.353237
2018-10-01    51.592274
2018-10-02 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


49.1962953296073
[49.14781883 49.4281419  47.95005738 49.90925425 49.54620428]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")